## Auto label data using foundation models

This notebook is adapted from [here](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/how-to-auto-train-yolov8-model-with-autodistill.ipynb).

In [ ]:
# Do once
%pip install -q \
 autodistill \
 supervision

### Display image sample

In [ ]:
import os
import supervision as sv

HOME = os.getcwd()
IMAGE_DIR_PATH = f"{HOME}/images"

image_paths = sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["png", "jpg"])

print('image count:', len(image_paths))

Plot sample of our image dataset.

In [ ]:
SAMPLE_SIZE = 16
SAMPLE_GRID_SIZE = (4, 4)
SAMPLE_PLOT_SIZE = (16, 16)

In [ ]:
# import cv2
# import supervision as sv

# titles = [
#     image_path.stem
#     for image_path
#     in image_paths[:SAMPLE_SIZE]]
# images = [
#     cv2.imread(str(image_path))
#     for image_path
#     in image_paths[:SAMPLE_SIZE]]

# sv.plot_images_grid(images=images, titles=titles, grid_size=SAMPLE_GRID_SIZE, size=SAMPLE_PLOT_SIZE)

### Autolabel images

Here we define ontology.

**Ontology** - an Ontology defines how your Base Model is prompted, what your Dataset will describe, and what your Target Model will predict. A simple Ontology is the CaptionOntology which prompts a Base Model with text captions and maps them to class names.

In [ ]:
from autodistill.detection import CaptionOntology

# left -> prompt sent to model (should be descriptive)
# right -> class label we want the model to predict

# description from ChatGPT
# prompt: describe what a {class name} looks like in 15-20 words

ontology=CaptionOntology({
    "a person is a living being with a complex physical form including a head torso limbs and varied appearance based on ethnicity and individual traits": "person",
    "a rickshaw is a human powered or motorized vehicle with a simple frame seating and often two or three wheels" : "rickshaw",
    "a rickshaw van is a motorized three wheeled vehicle with an enclosed cabin for passengers or goods and typically a driver upfront": "rickshaw van",
    "an auto rickshaw is a compact three wheeled motorized vehicle with a cabin for passengers a driver upfront and a rear engine": "auto rickshaw",
    "a truck is a large motorized vehicle with a drivers cabin cargo area wheels and often a distinct front grille": "truck",
    "a pickup truck is a smaller motorized vehicle with a drivers cabin and an open cargo bed in the rear": "pickup truck",
    "a private car is a four wheeled motor vehicle designed for personal transportation typically with seating for passengers and an enclosed cabin": "private car",
    "a motorcycle is a two wheeled motor vehicle with a seat for a rider and often a pillion seat for a passenger": "motorcycle",
    "a bicycle is a human powered vehicle with two wheels pedals a frame handlebars and a seat for a rider": "bicycle",
    "a bus is a large motorized vehicle with a passenger cabin typically featuring multiple seats windows and a distinctive elongated shape": "bus",
    "a micro bus is a smaller motorized vehicle similar to a standard bus but more compact with seating for fewer passengers": "micro bus",
    "a covered van is a motorized vehicle with a closed cargo area often used for transporting goods and may have a drivers cabin upfront": "covered van",
    "a human hauler is a motorized vehicle designed for transporting passengers similar to an auto rickshaw or tuktuk with a cabin and driver upfront": "human hauler"
})

### Initiate base model and autolabel

**Base Model** - A Base Model is a large foundation model that knows a lot about a lot. We use a Base Model (along with unlabeled input data and an Ontology) to create a Dataset.

In [ ]:
# # DETIC
# from autodistill_detic import DETIC
# DATASET_DIR_PATH = f"{HOME}/bdss_detic"
# base_model = DETIC(ontology=ontology)
# dataset = base_model.label(
#     input_folder=IMAGE_DIR_PATH,
#     extension=".jpg",
#     output_folder=DATASET_DIR_PATH)

# # Grounding DINO
from autodistill_grounding_dino import GroundingDINO
DATASET_DIR_PATH = f"{HOME}/bdss_gdino"
base_model = GroundingDINO(ontology=ontology)
dataset = base_model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".jpg",
    output_folder=DATASET_DIR_PATH)